# $\color{red}{\text{DL Final Project: MRI Tumor Detection}}$

# NEW

<h1 style="background-color:DodgerBlue;">
Necessary imports and Load dataset
</h1>



In [1]:
# Import necessary libraries
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h2 style="background-color:DodgerBlue;">
Load dataset, review, and split to train and test
</h2>



In [2]:
# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Function to count images in a folder
def count_images(folder_path, prefix):
    count = 0
    for file_name in os.listdir(folder_path):
        if file_name.startswith(prefix):
            count += 1
    return count

# Specify the directory path
dir_path = '/kaggle/input/mritumor/Data'

# categories = ['Glioma Tumor', 'Meningioma Tumor', 'Normal', 'Pituitary Tumor']

# Find categories (folders) in the specified directory
categories = [folder_name for folder_name in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, folder_name))]

# Initialize counts for each category
category_counts = {category: 0 for category in categories}

# Iterate through the folders and count images
for category in categories:
    category_path = os.path.join(dir_path, category)
    category_counts[category] = count_images(category_path, '')

# Print the counts for each category
for category, count in category_counts.items():
    print(f'Number of images in {category}: {count}')

Number of images in pituitary_tumor: 844
Number of images in meningioma_tumor: 913
Number of images in glioma_tumor: 901
Number of images in normal: 438


In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ... (previous code)

# Split the data into training, validation, and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=test_percent, random_state=seed, stratify=labels
)

train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=val_percent/(1-train_percent), random_state=seed, stratify=train_labels
)

# Create generators for training, validation, and testing sets
batch_size = 32  # Adjust as needed
img_size = (150, 150)  # Adjust as needed

train_gen = datagen.flow(
    x=train_files,
    y=train_labels,
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

val_gen = datagen.flow(
    x=val_files,
    y=val_labels,
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

test_gen = datagen.flow(
    x=test_files,
    y=test_labels,
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)


TypeError: ImageDataGenerator.flow() got an unexpected keyword argument 'target_size'

In [3]:
# Specify the percentage of data for training, validation, and testing
train_percent = 0.8  # 80% for training
val_percent = 0.1    # 10% for validation
test_percent = 0.1   # 10% for testing

# Create empty lists to store file paths and corresponding labels
file_paths = []
labels = []

# Iterate through the folders and collect file paths and labels
for category in categories:
    category_path = os.path.join(dir_path, category)
    image_files = [file_name for file_name in os.listdir(category_path) if file_name.endswith('.jpg')]
    file_paths.extend([os.path.join(category_path, file_name) for file_name in image_files])
    labels.extend([category] * len(image_files))

# Split the data into training, validation, and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=test_percent, random_state=seed, stratify=labels
)

train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=val_percent/(1-train_percent), random_state=seed, stratify=train_labels
)

# Print the number of samples in each set
print(f'Number of training samples: {len(train_files)}')
print(f'Number of validation samples: {len(val_files)}')
print(f'Number of testing samples: {len(test_files)}')


Number of training samples: 1392
Number of validation samples: 1394
Number of testing samples: 310


In [ ]:
# # Set up output directories
# output_dir = '/kaggle/working/dataset'
# test_size = 0.2
# train_dir = os.path.join(output_dir, 'train')
# test_dir = os.path.join(output_dir, 'test')
# val_dir = os.path.join(output_dir, 'val')

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)

# # Split images into train, test, and val sets
# for category in categories:
#     category_path = os.path.join(dir_path, category)
#     images = [file_name for file_name in os.listdir(category_path) if file_name.endswith('.jpg')]
#     train_images, test_val_images = train_test_split(images, test_size=test_size, random_state=seed)
#     test_images, val_images = train_test_split(test_val_images, test_size=0.5, random_state=seed)

#     for image in train_images:
#         src_path = os.path.join(category_path, image)
#         dest_path = os.path.join(train_dir, category, image)
#         os.makedirs(os.path.dirname(dest_path), exist_ok=True)
#         shutil.copyfile(src_path, dest_path)

#     for image in test_images:
#         src_path = os.path.join(category_path, image)
#         dest_path = os.path.join(test_dir, category, image)
#         os.makedirs(os.path.dirname(dest_path), exist_ok=True)
#         shutil.copyfile(src_path, dest_path)

#     for image in val_images:
#         src_path = os.path.join(category_path, image)
#         dest_path = os.path.join(val_dir, category, image)
#         os.makedirs(os.path.dirname(dest_path), exist_ok=True)
#         shutil.copyfile(src_path, dest_path)

# # Print the number of images in each set
# print(f'\nNumber of training images: {int(sum(category_counts.values()) * (1 - test_size))}')
# print(f'Number of testing images: {int(sum(category_counts.values()) * test_size)}')
# print(f'Number of validation images: {int(sum(category_counts.values()) * test_size * 0.5)}')

In [4]:
def preprocess_images(images):
    preprocessed_images = []
    for i in range(len(images)):
        image = images[i]
        
        # Convert the BGR image to LAB color space
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         # Split the LAB image into L, A, and B channels
        r, g, b = cv2.split(img)
#         print(r)
    
        # Ensure the channels have the correct data type (8-bit unsigned)
        r = r.astype(np.uint8)
        g = g.astype(np.uint8)
        b = b.astype(np.uint8)
        
        # Apply CLAHE to each channel separately
        clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8,8))
        r = clahe.apply(r)
        g = clahe.apply(g)
        b = clahe.apply(b)

        # Merge the enhanced RGB channels 
        img_output = cv2.merge([r, g, b])
        
       
        # Convert the LAB image back to BGR color space
        preprocessed_image = cv2.cvtColor(img_output, cv2.COLOR_RGB2BGR)
        
        preprocessed_images.append(preprocessed_image)
            
    return np.array(preprocessed_images)

In [5]:
# Constants
img_size = (256, 256)
batch_size = 32
img_shape = (img_size[0], img_size[1], 3)

In [6]:
# ImageDataGenerator with data augmentation
tr_gen = ImageDataGenerator(
    rescale=1./255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
#     preprocessing_function=preprocess_images  # Custom preprocessing function using a median filter
)

# Image data generator for loading and augmenting images
train_gen = tr_gen.flow_from_directory(
    train_dir, 
    target_size=img_size, 
    class_mode='categorical',
    color_mode='rgb', 
    shuffle=True, 
    batch_size=batch_size
)

# Create ImageDataGenerator without data augmentation
datagen = ImageDataGenerator(
    rescale=1./255.0,  # Rescale pixel values to the range [0, 1]
)


# Validation data generator
valid_gen = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size
)

# Test data generator
test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=False,
    batch_size=batch_size
)

NameError: name 'train_dir' is not defined

In [ ]:
# Retrieve a batch of images and labels from the generator
images, labels = next(train_gen)
num_samples = len(images)

# Plot the images with class names
plt.figure(figsize=(20, 20))
for i in range(min(16, num_samples)):
    plt.subplot(4, 4, i + 1)
    # Explicitly cast to float32 before normalization
    image = images[i].astype('float32') / 255.0  

    plt.imshow(image)
    class_index = int(labels[i])
    class_name = classes[class_index]  # Assuming you have defined 'classes' based on your dataset
    plt.title(class_name, color='blue', fontsize=12)
    plt.axis('off')
plt.show()


<h2 style="background-color:red;">
<!-- <div style="direction:rtl">  </div> -->
Making Model 
</h2>



In [ ]:
# Model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=img_shape, pooling='max')
base_model.trainable = False

num_classes = len(categories)

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=tf.keras.regularizers.l2(l=0.016),
          activity_regularizer=tf.keras.regularizers.l1(0.006),
          bias_regularizer=tf.keras.regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.4, seed=seed),
    Dense(num_classes, activation='softmax')
])

<h2 style="background-color:red;">
<!-- <div style="direction:rtl">  </div> -->
Compile and run the Model 
</h2>



In [ ]:
model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Model training
history = model.fit(train_gen, validation_data=valid_gen, epochs=50, verbose=1)

<h2 style="background-color:Violet;">
<!-- <div style="direction:rtl">  </div> -->
Evaluate model
 </h2>



In [ ]:
# Model evaluation
train_score = model.evaluate(train_gen, steps=len(train_gen))
val_score = model.evaluate(valid_gen, steps=len(valid_gen))
test_score = model.evaluate(test_gen, steps=len(test_gen))

print(f'Train loss = {train_score[0]}')
print(f'Train Accuracy = {train_score[1]}')
print(f'Validation loss = {val_score[0]}')
print(f'Validation Accuracy = {val_score[1]}')
print(f'Test loss = {test_score[0]}')
print(f'Test Accuracy = {test_score[1]}')

In [ ]:
# Model predictions
pred_probs = model.predict(test_gen)
predicted_classes = np.argmax(pred_probs, axis=1)

# Get true labels from the generator
true_classes = test_gen.classes

# Confusion matrix
confusion = confusion_matrix(true_classes, predicted_classes)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=test_gen.class_indices)
disp.plot(cmap='Blues', values_format='d')

# Classification report
classification_rep = classification_report(true_classes, predicted_classes, target_names=categories)
print("\nClassification Report:")
print(classification_rep)

In [ ]:
# Plotting accuracy and loss over epochs
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()
